# Getting Started with PyPSA

This tutorial introduces you to PyPSA (Python for Power System Analysis), a free software toolbox for simulating and optimizing modern power systems.

## Prerequisites

Before starting this tutorial, you should have:
- Basic understanding of Python programming
- Familiarity with power systems concepts (AC power flow, per-unit system)
- Python 3.7 or higher installed
- Understanding of pandas and numpy for data manipulation

## Installation

You can install PyPSA and its dependencies using pip:
```bash
pip install pypsa pandas numpy matplotlib
```

For development and type checking, you might also want:
```bash
pip install types-pandas types-numpy
```

## What is PyPSA?

PyPSA is a free software toolbox for simulating and optimizing modern power systems. It supports:

- Conventional generators with unit commitment
- Variable renewable generation (wind, solar)
- Storage units (batteries, pumped hydro)
- Transmission lines and networks
- Coupling to other energy sectors (heat, transport)
- Investment and operational optimization

## Basic Network Creation

Let's start by creating a simple network with three buses connected in a ring. This example will demonstrate basic PyPSA concepts.


In [ ]:
import pypsa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from typing import Optional

# Create a new network
network = pypsa.Network()

# Add three buses with nominal voltage of 20 kV
# Note: All voltages in PyPSA are in kV
for i in range(3):
    network.add("Bus", f"bus {i}", v_nom=20.0)

print("Buses in the network:")
print(network.buses)


## Adding Time Series

PyPSA works with time series data to model temporal variations in power systems. 
The snapshots define the time points for which the network will be analyzed.


In [ ]:
# Create time series for 24 hours
snapshots = pd.date_range("2024-01-01", periods=24, freq="h")
network.set_snapshots(snapshots)

print("Network snapshots:")
print(network.snapshots)


## Adding Basic Components

Now let's add some basic components to our network. We'll create:
- A gas generator at bus 0
- A load at bus 1
- Transmission lines connecting the buses in a ring

Note: All power values in PyPSA are in MW, and all costs are in EUR/MWh


In [ ]:
# Add a gas generator at bus 0 with 100 MW capacity
network.add(
    "Generator",
    "gas",
    bus="bus 0",
    p_nom=100,  # Nominal power in MW
    marginal_cost=50,  # Marginal cost in EUR/MWh
    carrier="gas",
    p_min_pu=0.3,  # Minimum power output as fraction of p_nom
    p_max_pu=1.0   # Maximum power output as fraction of p_nom
)

# Add a load at bus 1 with 50 MW demand
network.add(
    "Load",
    "load",
    bus="bus 1",
    p_set=50  # Fixed power demand in MW
)

# Add lines connecting the buses in a ring
# Note: Line parameters are in per-unit (p.u.) values
for i in range(3):
    network.add(
        "Line",
        f"line {i}",
        bus0=f"bus {i}",
        bus1=f"bus {(i + 1) % 3}",
        x=0.1,  # Reactance in p.u.
        r=0.01,  # Resistance in p.u.
        s_nom=100,  # Nominal apparent power in MVA
        s_max_pu=1.0  # Maximum loading as fraction of s_nom
    )

print("\nGenerators in the network:")
print(network.generators)

print("\nLoads in the network:")
print(network.loads)

print("\nLines in the network:")
print(network.lines)


## Basic Network Analysis

Let's perform a simple power flow analysis to understand how power flows through our network.
The power flow analysis solves for:
- Bus voltages and angles
- Active and reactive power flows
- Generator dispatch


In [ ]:
try:
    # Solve power flow
    network.pf()
    
    # Print power flows on lines
    print("Power flows on lines (positive values indicate flow from bus0 to bus1):")
    print(network.lines_t.p0)
    
    # Print bus voltages
    print("\nBus voltages (p.u.):")
    print(network.buses_t.v_mag_pu)
    
except Exception as e:
    print(f"Error during power flow calculation: {str(e)}")


## Visualization

Let's create a simple visualization of our network:


In [ ]:
try:
    # Plot the network
    network.plot()
    plt.title("Simple Three-Bus Network")
    plt.xlabel("Longitude")
    plt.ylabel("Latitude")
    plt.grid(True)
    plt.show()
except Exception as e:
    print(f"Error during visualization: {str(e)}")


## Key Concepts

1. **Network**: The main container for all components
2. **Bus**: A node in the network where components are connected
3. **Components**: Different types of equipment (generators, loads, lines, etc.)
4. **Time Series**: Snapshots represent different time periods
5. **Power Flow**: Analysis of power flows in the network

## Common Parameters

- `p_nom`: Nominal power capacity (MW)
- `marginal_cost`: Cost per unit of energy (EUR/MWh)
- `p_set`: Fixed power demand or generation (MW)
- `v_nom`: Nominal voltage (kV)
- `x`: Line reactance (p.u.)
- `r`: Line resistance (p.u.)
- `s_nom`: Nominal apparent power (MVA)
- `p_min_pu`: Minimum power output as fraction of p_nom
- `p_max_pu`: Maximum power output as fraction of p_nom
- `s_max_pu`: Maximum line loading as fraction of s_nom

## Next Steps

In the next tutorial, we'll explore:
- Adding variable renewable generation (wind, solar) with time series
- Implementing storage systems with charging/discharging constraints
- Creating more complex network topologies with multiple voltage levels
- Performing economic optimization with investment decisions
- Analyzing network constraints and congestion
- Adding contingency analysis and N-1 security checks
